In [2]:
import networkx as nx
import numpy as np
import pandas as pd
from tracktour import load_tiff_frames
from tracktour._graph_util import filter_to_migration_sol
from tracktour._viz_util import *
import napari
from napari_graph import DirectedGraph
from napari.layers import Graph

In [3]:
seg = load_tiff_frames('/home/draga/PhD/data/cell_tracking_challenge/ST_Segmentations/PhC-C2DL-PSC/02_ST/SEG/')
data = load_tiff_frames('/home/draga/PhD/data/cell_tracking_challenge/ST_Segmentations/PhC-C2DL-PSC/02/')
truth = load_tiff_frames('/home/draga/PhD/data/cell_tracking_challenge/ST_Segmentations/PhC-C2DL-PSC/02_GT/TRA/')

In [4]:
sol = nx.read_graphml('/home/draga/PhD/code/repos/tracktour/experiments/output/ST/PhC-C2DL-PSC/02_RES/full_solution.graphml', node_type=int)
filter_to_migration_sol(sol)
# assign_track_id(sol)
node_df = pd.DataFrame.from_dict(sol.nodes, orient='index')
# sol_mask = mask_by_id(node_df, seg)

merges = [node for node in sol.nodes if sol.in_degree(node) > 1]
bad_parents = [node for node in sol.nodes if sol.out_degree(node) > 2]

merge_edges = [edge for node in merges for edge in sol.in_edges(node)]
merge_edges.extend([edge for node in merges for edge in sol.out_edges(node)])

KeyboardInterrupt: 

In [5]:
store_colour_vs_of_interest(sol, set(merges + bad_parents), 'silver', 'black', 'gray')

In [6]:
vertex_sol = nx.read_graphml('/home/draga/PhD/code/repos/tracktour/_misc/final_solution.graphml', node_type=int)
filter_to_migration_sol(vertex_sol)
vertices_node_df = pd.DataFrame.from_dict(vertex_sol.nodes, orient='index')

vertex_sol_merges = [node for node in vertex_sol.nodes if vertex_sol.in_degree(node) > 1]
vertex_sol_bad_parents = [node for node in vertex_sol.nodes if vertex_sol.out_degree(node) > 2]

In [9]:
import json
with open('../_misc/oracle.json', 'r') as f:
    oracle = json.load(f)

In [11]:
introduced_vs = [oracle[k]['v_info'][0] for k in oracle.keys() if oracle[k]['decision'] == 'introduce']

In [13]:
store_colour_vs_of_interest(vertex_sol, introduced_vs, 'springgreen', 'darkgreen', 'limegreen')
store_colour_vs_of_interest(vertex_sol, set(vertex_sol_merges + vertex_sol_bad_parents), 'coral', 'red', 'maroon', True)

In [14]:
coords_df = node_df[['t', 'y', 'x']]
merge_graph = DirectedGraph(edges=merge_edges, coords=coords_df)
layer = Graph(
    merge_graph, 
    out_of_slice_display=True,
    ndim=3, 
    # scale=(50, 1, 1), 
    size=5, 
    properties=node_df,
    face_color=list(nx.get_node_attributes(sol, "color").values()),
)

full_graph = DirectedGraph(edges=list(sol.edges.keys()), coords=coords_df)
full_layer = Graph(
    full_graph, 
    out_of_slice_display=True,
    ndim=3, 
    # scale=(50, 1, 1), 
    size=5, 
    properties=node_df,
    face_color=list(nx.get_node_attributes(sol, "color").values()),
)

In [15]:
vertices_coords = vertices_node_df[['t', 'y', 'x']]
vertices_graph = DirectedGraph(edges=list(vertex_sol.edges.keys()), coords=vertices_coords)
vertices_layer = Graph(
    vertices_graph, 
    out_of_slice_display=True,
    # ndim=3, 
    # scale=(50, 1, 1), 
    size=5, 
    properties=vertices_node_df,
    face_color=list(nx.get_node_attributes(vertex_sol, "color").values()),
)

In [17]:
viewer = napari.Viewer()
viewer.add_image(data)
viewer.add_labels(
    seg,
    # scale=(50, 1, 1), 
)
viewer.add_labels(
    sol_mask,
    name='Solution'
)
viewer.add_labels(truth)
viewer.add_layer(layer)
viewer.add_layer(full_layer)
# viewer.add_layer(fixed_layer)
viewer.add_layer(vertices_layer)

/home/draga/miniconda3/envs/napari-graph/lib/python3.9/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 20 pixels
  return func(*args, **kwargs)
/home/draga/miniconda3/envs/napari-graph/lib/python3.9/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 20 pixels
  return func(*args, **kwargs)
/home/draga/miniconda3/envs/napari-graph/lib/python3.9/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 20 pixels
  return func(*args, **kwargs)
  File "/home/draga/miniconda3/envs/napari-graph/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/draga/miniconda3/envs/napari-graph/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/home/draga/miniconda3/envs/napari-graph/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/

<Graph layer 'vertices_graph' at 0x7f26097f1850>

In [21]:
merges

[467,
 576,
 2314,
 2514,
 2667,
 3628,
 3872,
 4075,
 4111,
 4648,
 4788,
 5113,
 5501,
 6720,
 6769]

In [23]:
oracle

{'467': {'decision': 'introduce',
  'v_info': [8606, [9.0, 505.0, 907.0, 390]],
  'parent': None},
 '576': {'decision': 'introduce',
  'v_info': [8607, [11.0, 505.0, 907.0, 391]],
  'parent': None},
 '2314': {'decision': 'introduce',
  'v_info': [8608, [37.0, 435.0, 177.0, 392]],
  'parent': None},
 '2514': {'decision': 'introduce',
  'v_info': [8609, [39.0, 509.0, 906.0, 393]],
  'parent': None},
 '2667': {'decision': 'introduce',
  'v_info': [8610, [41.0, 507.0, 905.0, 394]],
  'parent': None},
 '3628': {'decision': 'introduce',
  'v_info': [8611, [51.0, 448.0, 184.0, 395]],
  'parent': None},
 '3872': {'decision': 'introduce',
  'v_info': [8612, [53.0, 448.0, 184.0, 396]],
  'parent': None},
 '4075': {'decision': 'introduce',
  'v_info': [8613, [55.0, 448.0, 184.0, 397]],
  'parent': None},
 '4111': {'decision': 'introduce',
  'v_info': [8614, [55.0, 473.0, 904.0, 398]],
  'parent': None},
 '4648': {'decision': 'introduce',
  'v_info': [8615, [60.0, 445.0, 180.0, 399]],
  'parent': 

In [31]:
big_edges = [edge for edge in sol.edges if sol.edges[edge]['flow'] > 1]

In [34]:
big_edges

[(5, 48),
 (6, 49),
 (17, 60),
 (21, 64),
 (22, 65),
 (23, 66),
 (30, 73),
 (34, 77),
 (39, 82),
 (48, 90),
 (60, 103),
 (64, 107),
 (65, 108),
 (66, 109),
 (73, 116),
 (77, 120),
 (82, 125),
 (90, 135),
 (103, 148),
 (108, 154),
 (109, 155),
 (116, 162),
 (120, 166),
 (125, 171),
 (148, 194),
 (154, 200),
 (162, 209),
 (166, 213),
 (171, 218),
 (200, 247),
 (209, 256),
 (213, 260),
 (218, 265),
 (256, 304),
 (260, 308),
 (265, 313),
 (304, 352),
 (308, 356),
 (352, 402),
 (356, 406),
 (402, 454),
 (406, 458),
 (454, 506),
 (458, 510),
 (576, 632),
 (632, 691),
 (2514, 2599),
 (2599, 2687),
 (2667, 2756),
 (2687, 2777),
 (2777, 2872),
 (2872, 2968),
 (2968, 3067),
 (3067, 3166),
 (4075, 4187),
 (4111, 4224),
 (4224, 4338),
 (4338, 4454),
 (4454, 4570),
 (4570, 4685),
 (4648, 4764),
 (4685, 4800),
 (4764, 4880),
 (4788, 4904),
 (4800, 4916),
 (4904, 5021),
 (4916, 5033),
 (5033, 5149),
 (5113, 5233),
 (5149, 5269),
 (5233, 5353),
 (5269, 5389),
 (5353, 5474),
 (5389, 5511),
 (5474, 5595